In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from lxml import etree

In [3]:
from typing import List, Tuple

In [4]:
import string, re

In [5]:
# %pip install pymystem3==0.1.10 
# Решил попробовать морфологический анализатор от Яндекса. Но у него, видимо, 
# какие-то проблемы с Колабом

In [6]:
%pip install pymorphy2

     |████████████████████████████████| 61kB 1.4MB/s 
     |████████████████████████████████| 8.2MB 4.7MB/s 


In [7]:
import pymorphy2

In [8]:
def chewable(s):                                                # Убираем лишнее 
  webs = '(\S+\.\S+\.?\S+[\s^\v]?)'
  nums = '(\d+[\s^\v]?)'
  re_webs = re.compile(webs)                                    # Убираем сслыки
  re_pnct = re.compile('[%s]' % re.escape(string.punctuation))  # Пунктуацию
  re_nums = re.compile(nums)                                    # Почему-то числа становились наиболее значимыми признаками - убираю
  s_dewebbed = re_webs.sub('', s)
  s_stripped = re_pnct.sub('', s_dewebbed)
  s_denummed = re_nums.sub('', s_stripped)
  m = pymorphy2.MorphAnalyzer()
  lemmas = [m.parse(w)[0].normal_form for w in s_denummed.split()]
  s_chewbl = ' '.join(lemmas)
  return s_chewbl

In [9]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = chewable(column.text)                                  #тут оно, собственно, и торчит
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [10]:
fXML = '/content/drive/My Drive/bank_train_2016.xml'

In [11]:
texts, labels = load_sentirueval_2016(fXML)

In [12]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 9392, number of labels is 9392.


In [13]:
import random

In [14]:
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

neutral => банк москва ипотечный кредит условие
neutral => johncorкредить сбербанк онлайн заявка
positive => санкция ес не нарушить финансовый устойчивость газпромбанк ampquotвведенный европейский союз в отношение ряд российский
neutral => за месяц текущий год сбербанк заработать млрд рубль
positive => первый канал visa и mastercard продолжить работать с втб россельхозбанк и банк москва в обычный режим
neutral => сбербанк кредит н новгород
neutral => российский альфагруппа находиться в шаг от приобретение raiffeisen райффайзный банк аваль
neutral => автокредит втб самара
neutral => сбербанк кредит получить в санкт петербург
neutral => банк втб казань потребительский кредит
neutral => сбербанк киб предоставить кредит на сумма млрд антипинский нпз бизнес
positive => по новый банковский программа вологжанин мочь избежать выплата неустойка ноябрь год сбербанк россия запустить программа в рамка к
neutral => корпоративный портфель дагестанский филиал россельхозбанк составить порядок млрд руб

In [15]:
positive_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'positive']
negative_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'negative']

In [16]:
for cur in positive_tweets[:5]: print(cur)

sawikshuster yevhens главное чтоб банк сбер и втб
самый выгодный автокредит в втб
легко можно получить денежный кредит ы втб банк
снижение процентный ставка по кредит на недвижимость сбербанк
в наш сбербанк прийти американец и попросить сфоткаться с они у банкомат d я ващий быть в шок и на фотка полюбома получиться смешной xd


In [17]:
for cur in negative_tweets[:5]: print(cur)

shtirliz zhukovaolga winzard tereshenkov обязательно про сбербанк написать временами похлеще интернет магазин финт выкручивать
umkaalfariusigorsechin втб и сбер точно вопросблокировка операция или запрет на кредит
канада ввести санкция против газпромбанк вэб новатэк и российский оборонный компания
rt letokot если сбер и втб попасть под санкция быть жопа
rt tvru канада ввести санкция против оборонный и сырьевой компания и банк рф среди они в частность газпромбанк внешэк…


In [18]:
from nltk import word_tokenize

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize)

In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
vectorizer.fit(texts)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x7fad1eef6598>,
                vocabulary=None)

In [23]:
print(vectorizer.get_feature_names()[0:20])

['a', 'abaradagan', 'aberadez', 'abetaaсбербанк', 'abncapital', 'abramovdeputat', 'abunyasha', 'abysov', 'adagamov', 'adairrasi', 'adaron', 'aday', 'adenisenko', 'adindex', 'adlarguan', 'admeru', 'admitad', 'adubter', 'adworningju', 'aeroflot']


In [24]:
print(len(vectorizer.get_feature_names()))

8451


In [25]:
X = vectorizer.transform(texts)

In [26]:
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [27]:
print(texts[0])

взять кредит тюмень альфа банк


In [28]:
print(X[0])

  (0, 1830)	1
  (0, 1969)	1
  (0, 2308)	1
  (0, 4045)	1
  (0, 7709)	1


In [29]:
print(vectorizer.get_feature_names()[6321])

процесс


In [32]:
print(vectorizer.get_feature_names()[8000])

фл


In [33]:
print(vectorizer.get_feature_names()[8000])

фл


In [34]:
from sklearn.feature_extraction.text import TfidfTransformer

In [35]:
transformer = TfidfTransformer().fit(X)

In [36]:
X_transformed = transformer.transform(X)

In [37]:
print(X_transformed[0])

  (0, 7709)	0.7412351690813022
  (0, 4045)	0.22236351392695303
  (0, 2308)	0.42291199134314233
  (0, 1969)	0.2572964198244949
  (0, 1830)	0.3950555552257762


In [38]:
print(vectorizer.get_feature_names()[8000])

фл


In [39]:
print(vectorizer.get_feature_names()[7199])

сортировка


In [40]:
tokens_with_IDF = list(zip(vectorizer.get_feature_names(), transformer.idf_))

In [41]:
for feature, idf in tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => a
9.454573 => abaradagan
9.454573 => aberadez
9.454573 => abetaaсбербанк
9.454573 => abncapital
9.454573 => abramovdeputat
9.454573 => abunyasha
9.454573 => abysov
9.454573 => adagamov
9.454573 => adairrasi
9.454573 => adaron
9.454573 => aday
9.454573 => adenisenko
9.454573 => adindex
9.454573 => adlarguan
9.454573 => admeru
9.454573 => admitad
9.454573 => adubter
9.454573 => adworningju
8.538282 => aeroflot


In [42]:
sorted_tokens_with_IDF = sorted(tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [43]:
for feature, idf in sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => a
9.454573 => abaradagan
9.454573 => aberadez
9.454573 => abetaaсбербанк
9.454573 => abncapital
9.454573 => abramovdeputat
9.454573 => abunyasha
9.454573 => abysov
9.454573 => adagamov
9.454573 => adairrasi
9.454573 => adaron
9.454573 => aday
9.454573 => adenisenko
9.454573 => adindex
9.454573 => adlarguan
9.454573 => admeru
9.454573 => admitad
9.454573 => adubter
9.454573 => adworningju
9.454573 => afyanik


In [44]:
from sklearn.feature_selection import SelectPercentile, chi2

In [45]:
selector = SelectPercentile(chi2, percentile=20)

In [46]:
selector.fit(X_transformed, labels)

SelectPercentile(percentile=20, score_func=<function chi2 at 0x7fad0e51cd08>)

In [47]:
selected_tokens_with_IDF = [tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]

In [48]:
print(len(selected_tokens_with_IDF))

1690


In [49]:
for feature, idf in selected_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => aberadez
9.454573 => abunyasha
9.454573 => adworningju
9.049108 => akselenc
9.454573 => alexigrey
9.454573 => alexсбербанк
4.905973 => alfabank
6.592372 => alfabankby
9.454573 => alushin
9.454573 => ampeuroмлрд
9.049108 => ampnbspть
8.761426 => ampquotбанк
9.454573 => ampquotвведенный
9.049108 => ampquotчерный
9.454573 => andreymalgin
9.454573 => andreytr
8.761426 => antimaydan
9.049108 => apple
9.454573 => arhangelmisha
9.454573 => artemycheros


In [50]:
selected_and_sorted_tokens_with_IDF = sorted(selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [51]:
for feature, idf in selected_and_sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => aberadez
9.454573 => abunyasha
9.454573 => adworningju
9.454573 => alexigrey
9.454573 => alexсбербанк
9.454573 => alushin
9.454573 => ampeuroмлрд
9.454573 => ampquotвведенный
9.454573 => andreymalgin
9.454573 => andreytr
9.454573 => arhangelmisha
9.454573 => artemycheros
9.454573 => bankitb
9.454573 => behlbride
9.454573 => bosacoworavu
9.454573 => bullethunter
9.454573 => buribedixuk
9.454573 => catspower
9.454573 => cgillenpiy
9.454573 => chumikov
